# Capstone Project - Interim Report

### 1) Importing the required libraries

In [3]:
from datetime import datetime
from datetime import timedelta
import time
start_time = time.time()

In [4]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

1.5.1


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import   pandas                 as      pd
import   numpy                  as      np
import   matplotlib.pyplot      as      plt
import   seaborn                as      sns
import   scipy.stats            as      stats
from     scipy                  import  stats
import   statsmodels.api        as     sm
from     sklearn.preprocessing                 import   LabelEncoder
from     statsmodels.stats.outliers_influence  import   variance_inflation_factor
import   warnings

from    sklearn.preprocessing    import   StandardScaler
from    sklearn.preprocessing    import   MinMaxScaler
from    sklearn.tree             import   DecisionTreeClassifier
from    sklearn.metrics          import   accuracy_score, explained_variance_score

from    sklearn.experimental     import enable_iterative_imputer
from    sklearn.impute           import IterativeImputer
from    sklearn.linear_model     import LinearRegression

In [7]:
from   sklearn.feature_selection   import  RFECV
from   sklearn.feature_selection   import  RFE
from   sklearn                     import  metrics, preprocessing
from   sklearn.pipeline            import  Pipeline
from   sklearn.model_selection     import  cross_val_score, RepeatedStratifiedKFold, train_test_split
 
from   sklearn.ensemble            import  RandomForestClassifier


In [8]:
import  os

folder =  r'E:\DSE-FT-B-JUN24-G2'
pd.set_option('display.max_columns', None)
os.chdir(folder)

In [9]:
os.getcwd()

'E:\\DSE-FT-B-JUN24-G2'

#### Function to calculate count and percentage of missing values

In [11]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero & Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero & Missing Values'] = 100 * mz_table['Total Zero & Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

In [12]:
def identify_outliers(df, var):
    # Interquartile Range (IQR)

    # Calculate the upper and lower limits
    Q1                     =    df[var].quantile(0.25)
    Q3                     =    df[var].quantile(0.75)
    IQR                    =    Q3 - Q1  
    low                    =    Q1 - (1.5 * IQR)
    
    lower                  =    abs(Q1 - 1.5 * IQR)
    upper                  =    abs(Q3 + (1.5 * IQR))
    outlier_upper          =    df[df[var] > upper].count()[1]
    outlier_lower          =    df[df[var] < low].count()[1]
    
    m                      =    np.min(df[var])
    mX                     =    np.max(df[var])
    Zero_l                 =    df.loc[df[var]==0,var].sum()
    
    print("\nFeature :{} Lower_quartile: {} ,Upper_quartile: {} ".format(var,lower, upper))
    print("\nMax value: {} Minimum value: {}".format(mX, m))
    print("\nGreater than UpperIQR : {}, Less than lower_IQR : {} ,  zero_count:  {}".format(outlier_upper,outlier_lower,Zero_l))
    
    outliers_present       =   outlier_upper > 0 or outlier_lower > 0
    if outliers_present:
       print("\nVariable {} has outliers".format(var))
    else:
       print("\nVariable {} has NO outliers".format(var))        
    df.boxplot(column = [var])
    plt.show()

In [13]:
def chk_ttest(var):
    g1           =    df.loc[df['Exited'] == 0, var]
    g2           =    df.loc[df['Exited'] == 1, var]    
    stat, pval   =    stats.ttest_ind(g1, g2, equal_var = False)
    txt          =    "\nThere is No statistically significant difference between the mean values of two groups of the Variable, {} ".format(var)
    if pval < 0.05:
       txt       =    "\nThere is a statistically significant difference between the mean values of two groups of the Variable, {} ".format(var)
    
    print(txt)

In [14]:
def chk_chisq(i, var):
    print("\n{0}: Variable, {1}".format(i, var))
    crosstab                                    = pd.crosstab(df[var], df['Exited'])
    res                                         = stats.chi2_contingency(crosstab)
    txt                                         =  "Independent Variable and Target variable are independent"
    print("\nThe important assumption: No more than 20% of the cells have and expected cell count < 5")
    print("\nThis can be checked by looking at the expected frequency table.")
    print(res)
    # Calculate the percentage of cells with expected counts less than 5\
    expected                =  res[3]
    percentage_low_expected = (expected < 5).sum().sum() / (expected.shape[0] * expected.shape[1]) * 100
    print("\n")
    print(f"Percentage of cells with expected counts less than 5: {percentage_low_expected:.2f}%")
    
    if res[1] < 0.05:
        txt                                     =  "Independent Variable,{} and Target variable are dependent".format(var)    
 
    print("\n{}".format(txt))   

In [15]:
def  hist_numerical(var, df):
    mean_mu   =   df[var].mean()
    ttile     =  "Histogram for the variable: {}".format(var)
    mu_txt    =  r'$\mu = $' + str(round(mean_mu,1)) 
    # Histogram
    sns.histplot(x= df[var], kde = True)
    plt.title(ttile)
    plt.xticks(rotation = 90)
    plt.show()

In [16]:
def  draw_countplot(df, i, var):
     print("\n{0}: Variable, {1}".format(i, var))
     x         =  df[var]
     y         =  df['Exited']
     ttile     =  "Bar Chart for the variable: {}".format(var) 
     plt.title(ttile)    
     sns.countplot(x = x, hue = y, data = df)
     plt.show()

In [17]:
def draw_boxplot1(df, i, var):
    print("\n{0}: Variable, {1}".format(i, var))
    titlex  =  '\n\nBoxplot for ' + var + '\n'
    sns.catplot(data = df, x = 'Geography', y = var, col = 'Gender', hue = 'Exited', kind = 'box')
    plt.xlabel(var)
    plt.ylabel('Geography')
    plt.show()

In [18]:
def draw_boxplot2(df, i, var):
    print("\n{0}: Variable, {1}".format(i, var))
    titlex  =  '\n\nBoxplot for ' + var + '\n'
    sns.catplot(data = df, x = 'Card Type', y = var, col = 'Gender', hue = 'Exited', kind = 'box')
    plt.xlabel(var)
    plt.ylabel('Card Type')
    plt.show()

In [19]:
def pointbiserialr(df, col1, y):
    x   =  df[col1]
    stat, p    =  stats.pointbiserialr(y, x)
    print('stat=%.3f, p=%.3f' % (stat, p))
    if p > 0.05:
     print('{} There is no correlation'.format(col1))
    else:
     print('{} There is a correlation'.format(col1))

In [20]:
def  label_enc(df, col):
     le        =  LabelEncoder()
     colx      =  col + '_code'
     df[colx]  = le.fit_transform(df[col])  

In [21]:
### compare_models
### ------------------------------------------------------------------------------------------

"""
9) Function Name: compare_models

   Description: This **function** compares various algorithms on 
                 1) AUROC 2) Precision, 3) Recall
   
   Input: 1) splits for k fold 
          2) random seed number
          3) Training data for predictor variables
          4) Training data for target variable



   Output: Model comparison on these metrics 1) AUROC 2) Metrics - Precision, Recall
   
"""
def compare_models(n_splits, random_state, X_train, Y_train):  

    ### To compare algorithms
    
    from    matplotlib                  import   pyplot
    from    sklearn.model_selection     import   KFold
    from    sklearn.model_selection     import   StratifiedKFold
    from    sklearn.model_selection     import   cross_val_score
    from    sklearn.linear_model        import   LogisticRegression
    from    sklearn.tree                import   DecisionTreeClassifier
    from    sklearn.neighbors           import   KNeighborsClassifier
    from    sklearn.naive_bayes         import   GaussianNB
    from    sklearn.ensemble            import   RandomForestClassifier
    
    ### Prepare models
    
    models  = []
    models.append(('LR', LogisticRegression()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeClassifier()))
    models.append(('NB', GaussianNB()))
    models.append(('RF', RandomForestClassifier()))
    
    
    ### Evaluate model in turn
    

    scores_req =  ['roc_auc', 'precision', 'recall']
   
    print("\n n_splits %d random_state %d" % (n_splits, random_state)) 
    
    for i in range(len(scores_req)):
        results    =   []
        names      =   []        
        scoring    =  scores_req[i]
        
        print(scoring)
        
        for name, model in models:

            ## kfold           =   KFold(n_splits = n_splits, random_state = 12345) 
            skf             =   StratifiedKFold(n_splits=n_splits, random_state = random_state, shuffle = True)
            cv_results      =   cross_val_score(model, X_train, Y_train, cv = skf, scoring = scoring)
            results.append(cv_results)	
            names.append(name)
            msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
            print(msg)
        
        ### Box plot algorithm comparison
        
        sub_title = 'Algorithm Comparison using ' + scoring
        
        fig = pyplot.figure()
        fig.suptitle(sub_title)
        ax  = fig.add_subplot(111)
        pyplot.boxplot(results)
        ax.set_xticklabels(names, rotation=90, ha='right')
        pyplot.show()

### ------------------------------------------------------------------------------------------

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh = 5):
    cols = X.columns
    variables = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
            variables = np.delete(variables, maxloc)
            dropped=True

    print('Remaining variables:')
    print(X.columns[variables])
    ncdf = X[cols[variables]]
    return ncdf

In [23]:
def get_significant_vars( lm, sig_level ):
    
    import pandas as pd
    
    var_p_vals_df         = pd.DataFrame( lm.pvalues )
    var_p_vals_df['vars'] = var_p_vals_df.index
    var_p_vals_df.columns = ['pvals', 'vars']
    
    return list( var_p_vals_df[var_p_vals_df.pvals <= sig_level]['vars'] )


In [24]:
def check_linearity(x1, df, title, y1):
    
    import matplotlib.pyplot as plt
    import seaborn           as sns
    
    sns.regplot(x = x1, y= y1, data= df, logistic= True).set_title(title)
    plt.show()

In [25]:
def logit_reg(n_splits, random_state, X_train, X_test, Y_train, Y_test ):  

    import statsmodels.api          as      sm
    
    from   sklearn.linear_model     import  LogisticRegression 
    from   sklearn.metrics          import  classification_report
    from   sklearn.metrics          import  confusion_matrix
    from   sklearn.model_selection  import  cross_val_score
    
    from   sklearn.model_selection  import  KFold 
    
    model = LogisticRegression() 

    model.fit(X_train, Y_train) 
    predicted_train    = model.predict(X_train) 
    matrix             = confusion_matrix(Y_train, predicted_train)
    print("\nTraining Data")
    print(matrix)
    draw_cm(Y_train, predicted_train )
    
    accuracy_train = model.score(X_train, Y_train) 
    print("Training Accuracy: %.3f%%" % (accuracy_train * 100.0))

    print("\nTesting Data")

    predicted_testing  = model.predict(X_test) 
    matrix             = confusion_matrix(Y_test, predicted_testing)
    print(matrix)
    draw_cm(Y_test, predicted_testing)
    
    accuracy_test      = model.score(X_test, Y_test) 
    print("Test Accuracy: %.3f%%" % (accuracy_test * 100.0))
    

    measures_train     = classification_report(Y_train, predicted_train) 
    print("\nTraining data")
    print(measures_train) 

    measures_test      = classification_report(Y_test, predicted_testing) 
    print("\nTesting data")
    print(measures_test) 
    
    kfold              = KFold(n_splits = n_splits, shuffle = True, random_state = random_state)
    scoring            = 'roc_auc' 

    auc_train          = cross_val_score(model, X_train, Y_train,  scoring=scoring) 
    print("\nTraining data")
    draw_roc( Y_train, predicted_train)    


    auc_test           = cross_val_score(model, X_test, Y_test,  scoring=scoring) 
    print("\nTesting data")    
    draw_roc( Y_test, predicted_testing)
    
    print("\nWith K fold cross validation")
    
    scoring           = 'accuracy'
    print("\nScoring:  %s" %scoring)
    cv_accuracy_train = cross_val_score(model, X_train, Y_train, cv = kfold,  scoring=scoring)
    print(cv_accuracy_train)
    print("\nAccuracy: %.3f (%.3f)" % (cv_accuracy_train.mean(), cv_accuracy_train.std()))
    
    scoring           = 'precision'
    print("\nScoring:  %s" %scoring)    
    cv_precision_train = cross_val_score(model, X_train, Y_train, cv = kfold,  scoring=scoring)
    print(cv_precision_train)
    print("\nPrecision: %.3f (%.3f)" % (cv_precision_train.mean(), cv_precision_train.std()))
  
    scoring           = 'recall'
    print("\nScoring:  %s" %scoring)
    
    cv_recall_train   = cross_val_score(model, X_train, Y_train, cv = kfold,  scoring=scoring)
    print(cv_recall_train)
    print("\nRecall: %.3f (%.3f)" % (cv_recall_train.mean(), cv_recall_train.std()))
    
    scoring           = 'roc_auc'
    print("\nScoring:  %s" %scoring)    
    cv_roc_auc_train = cross_val_score(model, X_train, Y_train, cv = kfold,  scoring=scoring)
    print(cv_roc_auc_train)
    print("\nAUROC: %.3f (%.3f)" % (cv_roc_auc_train.mean(), cv_roc_auc_train.std()))
    

In [26]:
def draw_cm( actual, predicted ):
    
    import matplotlib.pyplot as   plt
    import sklearn.metrics   as   metrics
    import seaborn           as   sns
    
    cm = metrics.confusion_matrix( actual, predicted )
    sns.heatmap(cm, annot=True, fmt='.2f', xticklabels = ["Yes", "No"] , yticklabels = ["Yes", "No"] )
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()   

In [27]:
def draw_roc( actual, probs ):

    import pandas            as   pd
    import numpy             as   np
    import seaborn           as   sns
    import matplotlib.pyplot as   plt
    import sklearn.metrics   as   metrics
    import seaborn           as   sns
    
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
    drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(6, 4))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    plt.show()
    return fpr, tpr, thresholds

### 2) Read data

In [29]:
file   =  './Data/DSEFTB-JUN24-G2-EDA-Data_2024_11_23_18_25_55.csv'
df     =  pd.read_csv(file, encoding = 'Latin-1')

In [30]:
df.head()

,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,Target,race,payer_code,medical_specialty,diag_1,diag_2,diag_3
0,Female,[0-10),6,25,1,1,41,0,1,0,0,0,1,Nil,Nil,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,Caucasian,MD,Pediatrics-Endocrinology,250.83,345,224
1,Female,[10-20),1,1,7,3,59,0,18,0,0,0,9,Nil,Nil,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,0,Caucasian,MC,Hematology/Oncology,276,250.01,121
2,Female,[20-30),1,1,7,2,11,5,13,2,0,1,6,Nil,Nil,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,0,AfricanAmerican,HM,ObstetricsandGynecology,648,250,766
3,Male,[30-40),1,1,7,2,44,1,16,0,0,0,7,Nil,Nil,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,0,Caucasian,MC,InternalMedicine,8,250.43,248
4,Male,[40-50),1,1,7,1,51,0,8,0,0,0,5,Nil,Nil,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,0,Caucasian,MD,Hematology/Oncology,197,157,86


In [31]:
df['diag_1']    =   df['diag_1'].astype('category')
df['diag_2']    =   df['diag_2'].astype('category')
df['diag_3']    =   df['diag_3'].astype('category')

### 3) Understanding Dataset and Domain

### Variable categorization (count of numeric and categorical)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 47 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   gender                    101766 non-null  object  
 1   age                       101766 non-null  object  
 2   admission_type_id         101766 non-null  int64   
 3   discharge_disposition_id  101766 non-null  int64   
 4   admission_source_id       101766 non-null  int64   
 5   time_in_hospital          101766 non-null  int64   
 6   num_lab_procedures        101766 non-null  int64   
 7   num_procedures            101766 non-null  int64   
 8   num_medications           101766 non-null  int64   
 9   number_outpatient         101766 non-null  int64   
 10  number_emergency          101766 non-null  int64   
 11  number_inpatient          101766 non-null  int64   
 12  number_diagnoses          101766 non-null  int64   
 13  max_glu_serum             101

In [35]:
df.select_dtypes(include = 'object').head()

,gender,age,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,race,payer_code,medical_specialty
0,Female,[0-10),Nil,Nil,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Caucasian,MD,Pediatrics-Endocrinology
1,Female,[10-20),Nil,Nil,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,Caucasian,MC,Hematology/Oncology
2,Female,[20-30),Nil,Nil,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,AfricanAmerican,HM,ObstetricsandGynecology
3,Male,[30-40),Nil,Nil,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,Caucasian,MC,InternalMedicine
4,Male,[40-50),Nil,Nil,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,Caucasian,MD,Hematology/Oncology


In [36]:
df.select_dtypes(include = 'number').head()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,Target
0,6,25,1,1,41,0,1,0,0,0,1,0
1,1,1,7,3,59,0,18,0,0,0,9,0
2,1,1,7,2,11,5,13,2,0,1,6,0
3,1,1,7,2,44,1,16,0,0,0,7,0
4,1,1,7,1,51,0,8,0,0,0,5,0


Inferences:
    
    -->Number of Catagorical Columns : 35
    
    -->Number of Numerical Columns : 12 of which one is our target variable

### b) Variable categorization (count of numeric and categorical)

In [73]:
df.select_dtypes(exclude = 'number').columns

Index(['gender', 'age', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'race', 'payer_code',
       'medical_specialty', 'diag_1', 'diag_2', 'diag_3'],
      dtype='object')

In [75]:
df.select_dtypes(include = 'number').columns

Index(['admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'Target'],
      dtype='object')

## I ---> Numerical variables

### There are 12 numerical variables and listed below:

### 1 ) admission_type_id
### 2 ) discharge_disposition_id
### 3 ) admission_source_id
### 4 ) time_in_hospital
### 5 ) num_lab_procedures
### 6 ) num_procedures
### 7 ) num_medications
### 8 ) number_outpatient
### 9 ) number_emergency
### 10 ) number_inpatient
### 11 ) number_diagnoses
### 12 ) Target
       

## II ---> Categorical variables

### There are 35 categorical variables and listed below:

### 1 ) gender
### 2 ) age
### 3 ) max_glu_serum
### 4 ) A1Cresult
### 5 ) metformin
### 6 ) repaglinide
### 7 ) nateglinide
### 8 ) chlorpropamide
### 9 ) glimepiride
### 10 ) acetohexamide
### 11 ) glipizide
### 12 ) glyburide
### 13 ) tolbutamide
### 14 ) pioglitazone
### 15 ) rosiglitazone
### 16 ) acarbose
### 17 ) miglitol
### 18 ) troglitazone
### 19 ) tolazamide
### 20 ) examide
### 21 ) citoglipton
### 22 ) insulin
### 23 ) glyburide-metformin
### 24 ) glipizide-metformin
### 25 ) glimepiride-pioglitazone
### 26 ) metformin-rosiglitazone
### 27 ) metformin-pioglitazone
### 28 ) change
### 29 ) diabetesMed
### 30 ) race
### 31 ) payer_code 
### 32 ) medical_specialty
### 33 ) diag_1
### 34 ) diag_2
### 35 ) diag_3

       

#### Pre Processing Data Analysis (count of missing/ null values, redundant columns, etc.)

<b>Null or Missing Values</b>

In [79]:
NA_df =  missing_zero_values_table(df)

Your selected dataframe has 47 columns and 101766 Rows.
There are 0 columns that have missing values.


### II) Redundant Columns

In [82]:
k = df.columns
for i in k:
    print('Column Name:',i)
    print('Number of Unique Variables in',i,'is: ',df[i].nunique())

Column Name: gender
Number of Unique Variables in gender is:  3
Column Name: age
Number of Unique Variables in age is:  10
Column Name: admission_type_id
Number of Unique Variables in admission_type_id is:  8
Column Name: discharge_disposition_id
Number of Unique Variables in discharge_disposition_id is:  26
Column Name: admission_source_id
Number of Unique Variables in admission_source_id is:  17
Column Name: time_in_hospital
Number of Unique Variables in time_in_hospital is:  14
Column Name: num_lab_procedures
Number of Unique Variables in num_lab_procedures is:  118
Column Name: num_procedures
Number of Unique Variables in num_procedures is:  7
Column Name: num_medications
Number of Unique Variables in num_medications is:  75
Column Name: number_outpatient
Number of Unique Variables in number_outpatient is:  39
Column Name: number_emergency
Number of Unique Variables in number_emergency is:  33
Column Name: number_inpatient
Number of Unique Variables in number_inpatient is:  21
Colu

As obvious, RowNumber and CustomerId are unique variables to every entry and do not assist our model in any way since they have 10,000 unique values. 
Similarly, Surname has 2932 unique values and it does not assist our model in any way.

### Redundant columns are RowNumber, CustomerId and Surname

### We must therefore drop them as they are redundant variables.


In [ ]:
drop_cols = ['RowNumber','CustomerId','Surname']

In [ ]:
df.drop(columns = drop_cols, axis = 1, inplace = True)

In [ ]:
cat_cols = df.select_dtypes(include = 'object').columns

In [ ]:
cat_cols

In [ ]:
df_numeric           =   df.select_dtypes(include = 'number')

In [ ]:
df_numeric.columns

In [ ]:
df_numeric_    =  pd.DataFrame(df_numeric)
type(df_numeric_)

In [ ]:
df_categorical   =   df.select_dtypes(include = 'object')
df_categorical.columns

In [ ]:
type(df_categorical)

#### Observations

We have now dropped our redundant column and our data is ready for further processing.

### d) Alternate sources of data that can supplement the core dataset (at least 2-3 columns)
1) Price Perception
2) Trust

### e) Project Justification -  Project Statement, Complexity involved, Project Outcome –Commercial, Academic or Social value

#### Project Statement

To develop a robust customer churn prediction model for the European MNC bank operating in France, Germany, and Spain. 

#### Complexity involved - Low

#### Project Outcome –Commercial, Academic or Social value - Commercial

# 3) Data Exploration (EDA)

### Converting and Labelling the Data

Since the majority of machine learning algorithms are created to operate with numerical data, categorical data is converted to numerical data using label encoding techniques. 

**Label Encoding** is a common technique for converting categorical variables into numerical values. Each unique category value is assigned a unique integer based on alphabetical or numerical ordering.

In [ ]:
df.select_dtypes(include = 'object').columns

In [ ]:
new_df            =   df.copy()
new_df['Exited']  =   df['Exited']

Variables ('Geography', 'Gender', 'Card Type') need to be encoded for further analysis and preprocessing.

In [ ]:
cat_newdf         =  new_df.select_dtypes(exclude=['int64','float64'])
cat_cols          =  cat_newdf.columns
for col in cat_cols:
    label_enc(new_df, col)

In [ ]:
new_df.columns

In [ ]:
new_df.head().T

### 3.1) Relationship between variables

### Numerical variable vs Target variable

In [ ]:
# Define the numerical and categorical columns for analysis
numerical_columns    = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard','IsActiveMember',\
                       'EstimatedSalary', 'Complain', 'Satisfaction Score', 'Point Earned']
categorical_columns = ['Exited']

for  var in numerical_columns :
     new_df.boxplot(var, by = 'Exited')
     plt.show()


### 3.2) Check for 
### 3.2.1) multi-collinearity

In [ ]:
X = new_df[['CreditScore', 'Age', 'Tenure', 'Balance','NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
            'Complain', 'Satisfaction Score', 'Point Earned', 'Geography_code', 'Gender_code', 'Card Type_code']]
Y = new_df['Exited']

In [ ]:
#Checking for multicolleniarty in our independant variables
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

vif_data

As visible, the following five (5) variables are having high **VIF** value greater than 5.

### Interpretation of VIF value

The higher the value, the greater the correlation of the variable with other variables. Values of more than 4 or 5 are sometimes regarded as being moderate to high, with values of 10 or more being regarded as very high.

1) CreditScore, 2) Age, 3) NumOfProducts, 4) Satisfaction Score and 5) Point Earned.	

We need to remove variables having greater than the threshold value of 5 from our dataset.

### 3.2.2) Distribution of variables

## a) Histogram for continuous numerical variable

### A histogram helps to understand the distribution of values in single continuous column

In [ ]:
num_vars             =    ['Age',  'Balance',  'EstimatedSalary']
for  var in num_vars:
     hist_numerical(var, df)    

### Observations

**None of the continuous numerical variables ('Age',  'Balance',  'EstimatedSalary') are normally distributed.**

## b) Bar charts for categorical variables

### A bar chart (aka bar graph, column chart) plots numeric values for levels of a categorical feature as bars.

In [ ]:
# Define the categorical columns for analysis
categorical_columns    =   ['Geography', 'Gender', 'Card Type']
i                      =   1
for var in categorical_columns :
    draw_countplot(new_df, i, var)
    i += 1

### Box charts for numerical variables grouped by categorical variables

In [ ]:
new_df.columns

In [ ]:
numerical_columns    = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'Point Earned']
i  =  1
###
for var in numerical_columns :
    draw_boxplot2(new_df, i, var)
    i += 1

In [ ]:
### 
i  =  1
for var in numerical_columns :
    draw_boxplot2(new_df, i, var)
    i += 1
###

In [ ]:
titlex  =  '\n\nBoxplot for age \n'
sns.catplot(data = df, x = 'Complain', y = 'Age', hue = 'Exited', kind = 'box')
plt.xlabel('Complain')
plt.ylabel('Age')
plt.show()

### 3.2.3) Presence of outliers and its treatment 

Outliers badly affect mean and standard deviation of the dataset. · It increases the error variance and reduces the power of statistical tests. By applying outlier treatment, machine learning practitioners can handle extreme values effectively. The primary goals of outlier treatment are: Identifying Outliers: Through various statistical methods, such as visualizations and mathematical approaches, outliers can be detected within a dataset.

We are interested to identify the outliers in our continuos numerical variables such as 'Age',  'Balance',  'EstimatedSalary' that affects the mean & standard deviation rather than the discrete numerical variables. Discrete variables are typically categorical, meaning they take on a limited number of values or categories. 

### However, if the outlier is physically possible you should consider it.

In [ ]:
for  var in num_vars:
     identify_outliers(df, var)

### Ob servations

We have checked the presence of outliers in the continuous numerical variables ('Age', 'Balance', 'EstimatedSalary').
Only the variable, Age is having the outlier. We observe that the maximum value is 92 and it is a valid & phisically possible value. **Hence, we will retain the outliers for the variable, Age**. 

### 3.2.4) Statistical significance of variables

a) Numerical variables

An unpaired t-test (also known as an independent t-test) is a statistical procedure that compares the averages/means of two independent or unrelated group of numerical variables to determine if there is a significant difference between the two.

Hypotheses are assumptions about reality whose validity is possible but not yet proven. Two hypotheses are always formulated that assert exactly the opposite. These two hypotheses are the null hypothesis and the alternative hypothesis.

Null hypothesis $H_0$	Alternative hypothesis $H_1$

There is no mean difference between the two groups in the population.

 Two population means are equal.
 The two groups are from the same population.
 $H_0$: μ1 = μ2

 Example: There is no difference between each variable of those who churned and who not churned.

There is a mean difference between the two groups in the population.

 The two population means are not equal.
 The two groups are not from the same population.
  $H_1$: μ1≠ μ2

There is mean difference between the two groups in the population.

In [ ]:
# Define the numerical and categorical columns for analysis
numerical_columns    = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard','IsActiveMember',\
                       'EstimatedSalary', 'Complain', 'Satisfaction Score', 'Point Earned']
for var in numerical_columns :
    chk_ttest(var)

### Observations

Based on the unpaired t test, we find that
* there is **a statistically significant difference** between the mean values of two groups of the Variable, listed below:
1) CreditScore 
2) Age
3) Balance 
4) NumOfProducts 
5) IsActiveMember 
6) Complain 

* there is **no statistically significant difference** between the mean values of two groups of the Variable, listed below:
1) Tenure
2) HasCrCard
3) EstimatedSalary
4) Satisfaction Score
5) Point Earned

### Shapiro-Wilk test is a test of normality, it determines whether the given sample comes from the normal distribution or not. 

Null hypothesis: $H_0$: Samples are drawn from normal distribution.
Alternative hypothesis: $H_1$: Samples are NOT drawn from normal distribution.        

In [ ]:
num_vars             =    ['Age',  'Balance',  'EstimatedSalary']
i                    =    1
for  var in num_vars:
     x          =   df[var]
     stat, pval =   stats.shapiro(x)
     print("\n\n{0}: P value of Shapiro test for {1} is {2}".format(i, var, pval))
     i         +=  1
     if pval < 0.05:
        print("\nSamples of {} are NOT from a normal distribution".format(var))    
     else:
        print("\nSamples of {} are from a normal distribution".format(var))        


### Observations

Based on the Shapiro test for normality, we observe the following:
'Age',  'Balance',  'EstimatedSalary' are **not normally** dustributed.

b) Categorical variables

The $\chi^2$ - (Chi Sqaure) test of independence analysis utilizes a cross tabulation table between the variables of interest r rows and c columns. 

Based on the cell counts, it is possible to test if there is a relationship, dependence, between the variables and to estimate the strength of the relationship. 

#### Assumptions

* The two samples are independent
* No expected cell count is = 0
* No more than 20% of the cells have and expected cell count < 5

### Hypothesis

Null hypothesis $H_0$: Variables are independent	

Alternative hypothesis $H_1$: Variables are NOT independent	

In [ ]:
# Define the categorical columns for analysis
categorical_columns    =   ['Geography', 'Gender', 'Card Type']
i  =  1
for var in categorical_columns :
    chk_chisq(i, var)
    i += 1

### Observations

We have checked each categorical independent variable with our target categorical variable using Chi Square test of independence. We observed the assumption for chi-square test of independence *(No more than 20% of the cells have and expected cell count < 5)* is satisfied.

**Each of the independent variables, 'Geography', 'Gender' and the target variable, Exited are dependent.**

**Independent variable, 'Card Type' and the target variable, Exited are independent.**

###  Calculate Correlation Between Continuous & Binary Target Variable

Point biserial correlation is used to calculate the correlation between a binary categorical variable (a variable that can only take on two values) and a continuous variable and has the following properties:

Point biserial correlation can range between -1 and 1.
For each group created by the binary variable, it is assumed that the continuous variable is normally distributed with equal variances.
For each group created by the binary variable, it is assumed that there are no extreme outliers.

The hypotheses for point biserial correlation thus result in:

Null hypothesis: The correlation coefficient r = 0 (There is no correlation)

Alternative hypothesis: The correlation coefficient r ≠ 0 (There is a correlation)

In [ ]:
Target            =  df['Exited']
colsn             =  ['Age',  'Balance',  'EstimatedSalary']
for  col1 in colsn:
    x   =   df[col1]
    y   =   df['Exited']
    pointbiserialr(df, col1, y)

### 3.2.5) class imbalance and its treatment

In [ ]:
b = Y.value_counts()
sns.barplot(x = b.index,y = b.values)

### Observations

As visible, our data is highly imbalanced. Imbalanced datasets can lead to a bias towards the majority class, as the model is trained on a majority of samples from the majority class. This can result in poor performance in the minority class.
Hence, we need to treat data imbalance.

One approach to addressing imbalanced datasets is to oversample the minority class. The simplest approach involves duplicating examples in the minority class, although these examples don’t add any new information to the model. Instead, new examples can be synthesized from the existing examples. This is a type of data augmentation for the minority class and is referred to as the Synthetic Minority Oversampling Technique, or **SMOTE** for short.

**Another method is under-sampling.**

Under-sampling balances the dataset by reducing the size of the abundant class. This method is used when quantity of data is sufficient. By keeping all samples in the rare class and randomly selecting an equal number of samples in the abundant class, a balanced new dataset can be retrieved for further modelling.

We prefer Over-sampling by SMOTE method.

# 4) Feature Engineering

## 4.1) Whether any transformations required

Data transformation is used when data needs to be converted to match that of the destination system.

**We have performed label encoding to make our data suitable for model building.**

## 4.2) Scaling the data

In Data Processing, we try to change the data in such a way that the model can process it without any problems. And Feature Scaling is one such process in which we transform the data into a better version. Feature Scaling is done to normalize the features in the dataset into a finite range.

The concept of standardization comes into picture when continuous independent variables are measured at different scales. https://www.listendata.com/2017/04/how-to-standardize-variable-in-regression.html

Data scaling is applied to numeric columns. In our dataset we have three continuous numerical columns:.

1) Age
2) Balance
3) EstimatedSalary

The script below filters these three columns and removes the remaining columns.

In [ ]:
data_numeric = new_df.filter(['Age',  'Balance',  'EstimatedSalary'], axis = 1)

data_numeric.head()

Let’s plot some statistical values for the columns in our dataset using the describe() method.

In [ ]:
data_numeric.describe()

### Robust scaling 

Both standard and robust scalers transform inputs to comparable scales. The difference lies in how they scale raw input values.

Standard scaling uses mean and standard deviation. Robust scaling uses median and interquartile range (IQR) instead.

Robust scaling answers a simple question. How far is each data point from the input’s median? 

### The fact that robust scaling uses median and IQR makes it resistant to outliers. 

In [ ]:
from sklearn.preprocessing import RobustScaler
 
robust_scaler =   RobustScaler()
cols          =   ['Age',  'Balance',  'EstimatedSalary']
robust_arr    =   data_numeric.to_numpy()
robust_scaler.fit(robust_arr)
# scale all data points using median and IQR
robust_scaled_data =  robust_scaler.transform(robust_arr)

In [ ]:
robust_df     =   pd.DataFrame(robust_scaled_data, columns = cols)

In [ ]:
robust_df.describe()

## 4.3) Feature selection 

## Automatically select the number of features

The RFE method is available via the RFE class in scikit-learn.

RFE is a transform. To use it, first the class is configured with the chosen algorithm specified via the “estimator” argument and the number of features to select via the “n_features_to_select” argument.

The algorithm must provide a way to calculate important scores, such as a decision tree. The algorithm used in RFE does not have to be the algorithm that is fit on the selected features; different algorithms can be used.

Once configured, the class must be fit on a training dataset to select the features by calling the fit() function. After the class is fit, the choice of input variables can be seen via the “support_” attribute that provides a True or False for each input variable.

It can then be applied to the dataset by calling the transform() function.

In [ ]:
###
### We need to remove the not-standardized 3 columns and add standardised 3 columns
###
new_df.drop('Age', axis = 1, inplace = True)
new_df.drop('Balance', axis = 1, inplace = True)
new_df.drop('EstimatedSalary', axis = 1, inplace = True)

['Age',  'Balance',  'EstimatedSalary']

new_df['Age']             =  robust_df['Age'] 
new_df['Balance']         =  robust_df['Balance'] 
new_df['EstimatedSalary'] =  robust_df['EstimatedSalary'] 


# Adding the target variable
new_df['Exited']          =  new_df['Exited']

In [ ]:
y             =    new_df['Exited']
X             =    new_df[['CreditScore', 'Tenure', 'NumOfProducts','HasCrCard', 'IsActiveMember',\
                            'Complain','Satisfaction Score', 'Point Earned', 'Geography_code',\
                           'Gender_code', 'Card Type_code', 'Age', 'Balance', 'EstimatedSalary']]                       
print('X dimension {}'. format(X.shape))
print('y dimension {}'. format(y.shape))

In [ ]:
new_df.columns

In [ ]:
# create pipeline
rfe                    =     RFE(estimator = RandomForestClassifier(), n_features_to_select = 10)
model                  =     RandomForestClassifier(random_state = 42)
pipeline               =     Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv                     =    RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)
n_scores               =    cross_val_score(pipeline, X, y, scoring = 'balanced_accuracy', cv = cv, n_jobs = -1, error_score = 'raise')


In [ ]:
# report performance
print('balanced_accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

### Observation

we can see the RFE that uses a Random Forest and selects 10 features and then fits a model on the selected features achieves a balanced accuracy of about 100 %. 

**Balanced accuracy** in binary and multiclass classification problems to deal with imbalanced datasets. It is defined as the average of recall obtained on each class.

Fit an RFE model on the whole dataset and selects five features, then reports each feature column index (0 to 9), whether it was selected or not (True or False), and the relative feature ranking.

The “support_” attribute reports true or false as to which features in order of column index were included and the “ranking_” attribute reports the relative ranking of features in the same order.

In [ ]:
# fit RFE
rfe.fit(X, y)
# summarize all features
for i in range(X.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
x_names   =   X.columns
for i in range(X.shape[1]):
     namex    =  x_names[i]
     print('Column: %d, Name: %s Selected %s, Rank: %.3f' % (i, namex, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
selected_X  =   []

In [ ]:
for i in range(X.shape[1]):
     namex    =  x_names[i]
     if (rfe.support_[i] == True):
         selected_X.append(namex)
         print('Column: %d, Name: %s Selected %s, Rank: %.3f' % (i, namex, rfe.support_[i], rfe.ranking_[i]))
print("\nSelected important features {}".format(selected_X))

In [ ]:
selected_X = ['CreditScore', 'Tenure', 'NumOfProducts', 'IsActiveMember', 'Complain', 'Point Earned', 'Geography_code', 'Age', 'Balance', 'EstimatedSalary']

### Observation

We have selected 10 features which are important in predicting the target variable.

**We shall use this data for our model building.**

In [ ]:
X_   =    X.loc[:, selected_X]

In [ ]:
X_.shape

In [ ]:
time_elapsed_secs     = time.time() - start_time

time_elapsed_msg = "Execution took: %s secs (Wall clock time)" % timedelta(seconds=round(time_elapsed_secs))

print(time_elapsed_msg)   

## 4.4) Dimensionality reduction

Since we have selected top 15 variables affecting the dependent variable, our dataset is Not Huge. We are not going to apply dimensionality reduction such as Principal Component Analysis or Factor Analysis etc.

We use PCA when you have high-dimensional data to reduce its dimensionality while preserving most of the variance, simplifying analysis and visualization.

# 5) Assumptions

## 5.1) Check for the assumptions to be satisfied for each of the models in 
o	Classification – Decision Tree, Random Forest, SVM, Bagged and boosted models

### Base model checking

In [ ]:
X_.isnull().sum()

In [ ]:
n_splits     = 10
random_state = 123456

compare_models(n_splits, random_state, X_, y)

## Assumptions Check for Logistic Regression

### Assumption 1 - Binary logistic regression requires the target / dependent variable to be binary. 

For a binary regression, the factor level 1 of the dependent variable should represent the desired outcome (such as Success etc..).

### Assumption 2 -  Only the meaningful variables should be included.

### Assumption 3 -The predictor variables should not be correlated to each other meaning the model should have little or no multicollinearity.

### Assumption 4 - The independent variables are linearly related to the log odds.

### Assumption 5 -  Logistic regression requires quite a large number of observations.

#### Assumption 1 - Binary logistic regression requires the target / dependent variable to be binary. 

In [ ]:
new_df["Exited"].value_counts()

### Observations

The target variable is categorical having 0 and 1 binary

#### Assumption 2 -  Only the meaningful variables should be included.

We have ensured that there are no unwanted variables selected for model building.

#### Assumption 3 -The predictor variables should not be correlated to each other meaning the model should have little or no multicollinearity.

In [ ]:
def show_VIF(X):
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    vif_df = pd.DataFrame({'Feature': X.columns, 'VIF': vif})
    vif_df.sort_values(by = ['VIF'], ascending = False, inplace = True)

    return(vif_df)

In [ ]:
show_VIF(X_)

### Observations

The following variables are highly colliear as their VIF values exceed the threshold value of 5:
* 1)  CreditScore	(14.057075)
* 2)  NumOfProducts	(7.833422)
* 3)  PointsEarned	(7.243992)

In [ ]:
calculate_vif_(X_, thresh = 5)

In [ ]:
X_collinear = X_.loc[:, ['Tenure', 'IsActiveMember', 'Complain', 'Point Earned','Geography_code', 'Age', 'Balance', 'EstimatedSalary']]

show_VIF(X_collinear)

### Observations

The following variables are non-collinear:
1) Point Earned
2) Tenure
3) IsActiveMember
4) Geography_code	
5) Complain
6) Age
7) Balance
8) EstimatedSalary

#### Assumption 4 - The independent variables are linearly related to the log odds.

We need to check the assumption of Independent variables are linearly related to the log odds.

One way to checking this is to plot the Independent variables in question and look for an S-shaped curve.

In [ ]:
df2           =  new_df
target        =  'Exited'
num_variables =  ['Tenure', 'IsActiveMember', 'Complain', 'Point Earned','Geography_code', 'Age', 'Balance', 'EstimatedSalary']
for i in range(len(num_variables)):
    title = num_variables[i] + '  Log odds linear plot'
    xvar  = num_variables[i]
    check_linearity(xvar,    df2, title, target)

#### Assumption 5 -  Logistic regression requires quite a large number of observations.

In [ ]:
#Number of events (cases where Response == 1)
num_events = new_df['Exited'].sum()

# Number of predictor variables (excluding 'Response')
num_predictors = len(X_.columns)

# Number of events per predictor variable
events_per_predictor = num_events / num_predictors

print("Number of events:", num_events)
print("Number of predictor variables:", num_predictors)
print("Events per predictor:", events_per_predictor)

**Explanation:**

We calculate the number of events by summing the 'Exited' column, which represents the cases where the outcome of interest occurs.

We calculate the number of predictor variables by counting the number of columns in the DataFrame and excluding the outcome variable.

We divide the number of events by the number of predictor variables to get the events per predictor.

We can then compare the calculated events per predictor with the recommended guideline of 10-20. If the ratio is below this guideline, it may indicate a potential violation of the assumption of a sufficiently large sample size.

### Observations:

With 2038 events and 10 predictor variables, the calculated number of events per predictor is approximately 203.8. This exceeds the commonly recommended guideline of having at least 10-20 events per predictor variable.

Inference: The dataset appears to meet the assumption of having a sufficiently large sample size for logistic regression.

Having a high number of events per predictor variable suggests that there should be adequate statistical power and precision in estimating the model parameters, enhancing the reliability of the logistic regression analysis. Therefore, the dataset likely provides a robust basis for fitting a logistic regression model and conducting statistical inference.

### Build a base model
We are interested in the measure, recall for minority class for measuring the model performance. 

**Average of Recall & std. deviation in parenthesis for the minority class** of each of the  models used for comparison are listed below:

#### 1) LR: 0.998037 (0.003254)
#### 2) NB: 0.998037 (0.003254)
#### 3) RF: 0.998037 (0.003254)
#### 4) Decision Tree (aka CART): 0.992642 (0.005025)
#### 5) KNN: 0.065754 (0.017023)

Based on the above models comparison. we are choosing **Logistic Regression** as our base model.
Logistic Regression models have high interpretability compared to most classification algorithms due to optimized feature coefficients. Refer: https://www.codecademy.com/learn/machine-learning-logistic-regression/modules/dspath-logistic-regression/cheatsheet


In [ ]:
start_time = time.time()

In [ ]:
## Split the data into train and test datasets
X_train, X_test, Y_train, Y_test  = train_test_split(X_, y, stratify = y,test_size = 0.30)

In [ ]:
vals, counts = np.unique(Y_test, return_counts = True)
print(vals, counts)

In [ ]:
print("\nShape: Total observations %d Total features %d" %(X_train.shape[0], X_train.shape[1]))

In [ ]:
logit = sm.Logit( Y_train, sm.add_constant( X_train ) )
lg    = logit.fit()

#### Report Psuedo R-square, model coefficients and p-value

In [ ]:
print(lg.summary())

**Observation**

We observe that the McFadden R square (Pseudo R square) is 98.20 % and the model fitness is very good.
This McFadden approach is one minus the ratio of two log likelihoods. The numerator is the log likelihood of the logit model selected and the denominator is the log likelihood if the model just had an intercept.

A goodness of fit using McFadden‟s pseudo r square (ρ^2) is used for fitting the overall model. McFadden suggested ρ^2 values of between 0.2 and 0.4 should be taken to represent a very good fit of the model (Louviere et al.,2000).
http://www.lifesciencesite.com/lsj/life1002/286_B01288life1002_2028_2036.pdf

### List the significant variables at 5% level of significance

In [ ]:
significant_vars = get_significant_vars( lg, sig_level = 0.05 )
print(significant_vars)

**Observation**

The following variables are significant at 5 % level of significance:

| SlNo | Significant variable |                      | 
| ---- | -------------------- | -------------------- |   
| 1 | IsActiveMember | Whether the customer is active or not.| 
| 2 | Complain |Whether the customer has complaint or not. | 
| 3 | Age | Age of the customer | 

### Get Odds ratio

In [ ]:
significant_vars

In [ ]:
ODDs_Ratio_df =  pd.DataFrame({'Important Variable' : lg.params.index, 'Log-odds' : lg.params.values })

In [ ]:
ODDs_Ratio_df   =  ODDs_Ratio_df.loc[ODDs_Ratio_df['Important Variable'].isin(significant_vars), :]

In [ ]:
ODDs_Ratio_df.drop(0, inplace = True)

In [ ]:
ODDs_Ratio_df['Odds Ratio'] =  np.exp(ODDs_Ratio_df['Log-odds'])
ODDs_Ratio_df.sort_values(by=['Odds Ratio'], ascending=False, inplace = True)

In [ ]:
ODDs_Ratio_df

### Odds Ratio Interpretation for significant variables

Holding other things constant:

| Slno | Inference |
| ----- | ------------------------------------- |
| 1 | The odds of customer churn are 567627 times the odds of customer retention when the customer has complaints.|
| 2 | The odds of customer churn are approximately 2.78 times the odds of customer retention for one unit increase in Age.|

In [ ]:
n_splits, random_state = 10, 12345
                     
print(logit_reg(n_splits, random_state, X_train, X_test, Y_train, Y_test ))

### To get the elapsed time, get the end time of processing

In [ ]:
logit_elapsed_time_secs = time.time() - start_time

logit_elapsed_time_msg = "LR model - Execution took: %s secs (Wall clock time)" % timedelta(seconds=round(logit_elapsed_time_secs))

print(logit_elapsed_time_msg)   

## END